# 🧠 Đánh giá Mô hình (Model Validation)

Bạn đã xây dựng được một mô hình.
Nhưng mà… **nó có thật sự tốt không?**

Trong bài học này, bạn sẽ học cách **sử dụng kỹ thuật “model validation”** để đo lường chất lượng mô hình.
Đây là **bước cốt lõi** giúp bạn **cải thiện mô hình liên tục** theo thời gian.

---

## 🔍 Model Validation là gì?

Khi xây dựng bất kỳ mô hình machine learning nào, bạn **luôn cần đánh giá** xem mô hình đó hoạt động tốt đến đâu.
Trong hầu hết các trường hợp, thước đo quan trọng nhất chính là **độ chính xác dự đoán (predictive accuracy)** —
nói cách khác, **dự đoán của mô hình có gần với giá trị thực tế không**?

---

## ⚠️ Sai lầm thường gặp

Rất nhiều người mắc phải lỗi khi đo độ chính xác của mô hình:
> Họ dùng **chính dữ liệu huấn luyện (training data)** để dự đoán,
> rồi **so sánh kết quả dự đoán** với **giá trị thật trong tập huấn luyện**.

Cách này **nghe có vẻ ổn**, nhưng **thực tế là sai** —
vì mô hình chỉ đang “ghi nhớ” dữ liệu đã thấy, chứ **không học được cách tổng quát hoá** cho dữ liệu mới.

Chúng ta sẽ thấy vấn đề này rõ hơn ở phần code tiếp theo,
nhưng trước hết hãy hiểu cách đo lường chất lượng mô hình.

---

## 📊 Tóm tắt chất lượng mô hình

Giả sử bạn có **10.000 ngôi nhà** với giá trị thật và giá trị dự đoán.
Nhìn từng con số một sẽ **vô nghĩa và không thực tế**.
Thay vào đó, ta cần **một con số duy nhất** để tóm tắt toàn bộ sai lệch — gọi là **“metric”**.

Có rất nhiều metric khác nhau,
nhưng ở đây ta sẽ bắt đầu với một cái rất phổ biến:
👉 **MAE (Mean Absolute Error)** – Sai số tuyệt đối trung bình.

---

## 🧮 Hiểu về MAE (Mean Absolute Error)

### 🔸 Khái niệm cơ bản
Sai số của từng mẫu (mỗi ngôi nhà) được tính như sau:

\[
\text{error} = \text{actual} - \text{predicted}
\]

Ví dụ:
- Một ngôi nhà thực tế có giá **150.000 USD**
- Mô hình dự đoán **100.000 USD**
→ Sai số là **50.000 USD**

---

### 🔸 MAE hoạt động như thế nào?

1. Lấy **giá trị tuyệt đối** của mỗi sai số (để loại bỏ dấu âm/dương).
2. Tính **trung bình** của tất cả các sai số đó.

Kết quả là một con số duy nhất biểu thị:
> “Trung bình, dự đoán của mô hình lệch khoảng X đơn vị so với thực tế.”

---

## ✅ Tóm lại
- **MAE** giúp bạn biết dự đoán của mô hình “sai trung bình bao nhiêu”.
- Càng nhỏ → mô hình càng chính xác.
- Để tính MAE, bạn cần một mô hình (đã được huấn luyện).
- Sau khi tính xong, bạn có thể dùng metric này để **so sánh các mô hình khác nhau**.

---

> 💡 Trong bước tiếp theo, ta sẽ xem cách tính MAE bằng Python
> và học cách chia dữ liệu thành *training set* và *validation set* để đánh giá đúng hơn.


In [1]:
import pandas as pd

melbourne_file_path = '../input/melb_data.csv'
melbourne_data = pd.read_csv(melbourne_file_path)

filtered_melbourne_data = melbourne_data.dropna(axis=0)

# Chọn các cột đặc trưng
melbourne_features = ['Rooms', 'Bathroom', 'Landsize', 'BuildingArea',
                      'YearBuilt', 'Lattitude', 'Longtitude']
X = filtered_melbourne_data[melbourne_features]

# Chọn mục tiêu dự đoán
y = filtered_melbourne_data.Price

# Khai báo mô hình
from sklearn.tree import DecisionTreeRegressor
melbourne_model = DecisionTreeRegressor(random_state=1)

# Huấn luyện mô hình
melbourne_model.fit(X, y)

DecisionTreeRegressor(random_state=1)

In [2]:
# Tính toán MAE
from sklearn.metrics import mean_absolute_error
predicted_home_prices = melbourne_model.predict(X)
mae = mean_absolute_error(y, predicted_home_prices)

# ⚠️ Vấn đề với "In-Sample Scores"

Thước đo mà ta vừa tính được gọi là **“in-sample score”** —
tức là bạn dùng **cùng một tập dữ liệu** để **huấn luyện mô hình** và **đánh giá mô hình**.

Vấn đề là…
👉 Cách này **nghe hợp lý** nhưng lại **dễ khiến bạn bị đánh lừa**.

---

## 🎨 Ví dụ: Màu cửa nhà

Hãy tưởng tượng trong **thị trường bất động sản thực tế**,
màu **cửa nhà** (door color) **không hề liên quan** gì đến **giá nhà**.

Nhưng trong **tập dữ liệu huấn luyện** mà bạn đang dùng,
ngẫu nhiên lại xảy ra chuyện:
> **Tất cả những ngôi nhà có cửa màu xanh lá cây đều cực kỳ đắt.**

Mô hình có nhiệm vụ **tìm ra các pattern (mẫu)** liên quan đến giá nhà,
nên nó sẽ học ngay rằng:
> “Cửa màu xanh lá → Nhà đắt tiền 💰”

---

## 🧠 Chuyện gì xảy ra tiếp theo?

- Trong **training data**, mô hình **dự đoán rất chính xác**,
  vì nó đã **học thuộc lòng** pattern này.
- Nhưng trong **thực tế**, màu cửa **không thật sự ảnh hưởng** đến giá nhà.
  Khi mô hình gặp **dữ liệu mới**, nó sẽ **dự đoán sai bét**.

---

## 🧩 Tại sao cần “Validation Data”

Giá trị thực sự của mô hình **nằm ở khả năng dự đoán dữ liệu mới**.
Vì thế, ta phải **đo hiệu suất của mô hình trên dữ liệu mà nó chưa từng thấy**.

Cách đơn giản nhất là:

1. **Chia dữ liệu ban đầu** thành 2 phần:
   - **Training data** → để huấn luyện mô hình
   - **Validation data** → để kiểm tra độ chính xác

2. **Huấn luyện mô hình** chỉ bằng *training data*.
3. **Đánh giá mô hình** bằng *validation data*.

Phần dữ liệu dùng để kiểm tra này được gọi là **validation set** (tập kiểm định).

---

> 💡 Nói ngắn gọn:
> - “In-sample” = đánh giá trên dữ liệu đã dùng để train → dễ bị **ảo tưởng**.
> - “Validation” = đánh giá trên dữ liệu mới → phản ánh **thực tế hơn**.


In [3]:
from sklearn.model_selection import train_test_split

train_X, val_X, train_Y, val_Y = train_test_split(X, y, random_state=0)

melbourne_model = DecisionTreeRegressor()

melbourne_model.fit(train_X, train_Y)

val_predictions = melbourne_model.predict(val_X)
print(mean_absolute_error(val_Y, val_predictions))

260593.37249838604
